# 🌾 AgriChat (TinyLlama) — Fixed Prompt Version

**Lightweight edition — runs on free Colab GPU (T4).**

Improvements in this notebook:
- Formats prompts for instruction-tuned models so the model answers (not echo).
- Friendly greeting and chat-history saved.
- Fallback: uses LoRA adapter if present, otherwise uses base TinyLlama.
- Short, crisp replies (1–2 sentences).

---

## 1️⃣ Install dependencies

In [ ]:
!pip install -q transformers peft bitsandbytes safetensors sentencepiece gradio accelerate

## 2️⃣ Launch AgriChat (Fixed prompt format)
This lightweight notebook starts chat instantly (no training required).

In [ ]:

import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch, os

base = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
adapter_path = "./outputs/lora_adapter"

def load_model():
    print("🔍 Loading base TinyLlama model...")
    model = AutoModelForCausalLM.from_pretrained(base, device_map="auto", trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(base)
    if os.path.exists(adapter_path):
        try:
            print("✅ Found LoRA adapter — loading fine-tuned weights...")
            model = PeftModel.from_pretrained(model, adapter_path)
        except Exception as e:
            print("⚠️ Failed to load adapter, continuing with base model. Error:", e)
    else:
        print("⚠️ No LoRA adapter found — using base TinyLlama model only.")
    model.eval()
    return tokenizer, model

tokenizer, model = load_model()

def craft_prompt(user_text):
    # Instruction-style formatting the model expects
    return f"""### Instruction:
{user_text}

### Response:
"""

def generate_reply(prompt, history):
    instr_prompt = craft_prompt(prompt)
    inputs = tokenizer(instr_prompt, return_tensors='pt', truncation=True, max_length=128).to('cuda')
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=60, do_sample=False, temperature=0.2)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # trim the instruction part if model echoes it; keep generated response portion
    if '### Response:' in text:
        text = text.split('### Response:')[-1].strip()
    # keep first 1-2 sentences
    sentences = [s.strip() for s in text.replace('\n',' ').split('.') if s.strip()]
    short = '. '.join(sentences[:2])
    if short and not short.endswith('.'):
        short += '.'
    history.append((prompt, short))
    return history, history

# --- Gradio Chat UI with history save ---
with gr.Blocks(title="AgriChat — TinyLlama (Fixed)") as demo:
    gr.Markdown("# 👋 Hi, I’m AgriChat — your assistant for agriculture and customer service topics!") 
    gr.Markdown("Ask agriculture-related or customer-service questions. Replies are short (1–2 sentences)." )
    chatbot = gr.Chatbot(label="AgriChat Conversation", height=400)
    msg = gr.Textbox(label="Ask something...", placeholder="Type your question here and press Enter")
    clear = gr.Button("Clear Chat")
    save = gr.Button("💾 Download Chat History")
    
    def save_chat(history):
        text = '\n'.join([f"You: {h[0]}\nAgriChat: {h[1]}" for h in history])
        path = "/content/agri_chat_history_fixed.txt"
        with open(path, "w", encoding='utf-8') as f:
            f.write(text)
        return path

    msg.submit(generate_reply, [msg, chatbot], [chatbot, chatbot])
    clear.click(lambda: [], None, chatbot, queue=False)
    save.click(save_chat, [chatbot], gr.File(label="Download your chat"))

demo.launch(share=False, inbrowser=False)


✅ **Instructions:**
1. Runtime → Change runtime type → **GPU** → Save.
2. Run the cells from top to bottom.
3. Chat once the UI appears. Use 💾 to download history.